In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from web3 import Web3

In [2]:
ankr_url = 'https://rpc.ankr.com/eth'

web3 = Web3(Web3.HTTPProvider(ankr_url))

print(web3.isConnected())

True


In [3]:
# web3.eth.get(12345)

AttributeError: 'Eth' object has no attribute 'get'

In [ ]:
transaction = web3.eth.get_transaction('0x82c09f32d30c75dae94a149100941f71fa1e6b8f1aa11695f92af20164555443')

In [ ]:
# # df = pd.read_csv('/tmp/export-0x1332cb754d445F07ee0a8279D8654fc889536253.csv',index_col=False)
# df = pd.read_csv('/tmp/export-0x0c4ece75800273bb516394b8a82b6b180b5f86c4.csv',index_col=False)
# print(df.shape)
# df.head()

In [ ]:
DATETIME = dt.datetime.now()
START_DATE = dt.datetime.strptime('2021-09-08', '%Y-%m-%d')

In [ ]:
# address = '0x1332cb754d445F07ee0a8279D8654fc889536253'
# currency = 'BNB'
address = '0x0c4ece75800273bb516394b8a82b6b180b5f86c4'
currency = 'ETH'


input_df_columns = df.columns.tolist()

columns = [
    'Date', 
    'Type', 
    'Buy',
    'Currency (Buy)',
    'Fiat value (Buy)',
    'Sell',
    'Currency (Sell)',
    'Fiat value (Sell)',
    'Fee',
    'Currency (Fee)',
    'Fiat value (Fee)',
    'Exchange',
    'Wallet',
    'Account',
    'Transfer-Code',
    'Comment'
]

In [ ]:
df.at[0, 'Value_OUT(%s)' % currency]

In [ ]:
df['Type'] = df['To'].map(lambda x: 'Deposit' if x.lower() == address.lower() else 'Withdraw')
df['Date'] = df['DateTime']

display(df['Type'].value_counts())
df.head()

In [ ]:
df = df[
    pd.to_datetime(df['Date']) > START_DATE
]

df.head()

In [ ]:
df['Fee'] = df['TxnFee(%s)' % currency]
df['Currency (Fee)'] = '%s' % currency
df.head()

In [ ]:
for col in columns:

    if col not in df.columns:
        df[col] = ""

for i, row in df.iterrows():
    
    df.at[i, 'Transfer-Code'] = row['Txhash']
    
    if row['Type'] == 'Withdraw':
        
        df.at[i, 'Currency (Sell)'] = '%s' % currency
        df.at[i, 'Sell'] = row['Value_OUT(%s)' % currency]
    
    elif row['Type'] == 'Deposit':
        
        df.at[i, 'Currency (Buy)'] = '%s' % currency
        df.at[i, 'Buy'] = row['Value_IN(%s)' % currency]
        
    else:
        print('HAN')

In [ ]:
out_df = df.drop(columns=input_df_columns, errors="ignore")[columns].sort_values('Date')


In [ ]:
out_df['Comment'] = 'Parsed via explorer CSV parser (%s) at %s' % (currency, DATETIME)

In [ ]:
out_df.head(5)

In [ ]:
out_df.to_clipboard(index=False,header=False)